<a href="https://colab.research.google.com/github/atick-faisal/MultiViewUNet-TAVI/blob/main/src/training/TAVI_CLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report


# Fix GDrive

In [2]:
print("Installing G-Drive ... ")
os.system("pip install -U --no-cache-dir gdown --pre > /dev/null")

Installing G-Drive ... 


0

# Downoad Data

In [3]:
os.system('gdown "1wYPkE9LbNKExhnNTj6Enr0AS_qb-Vthn"')
os.system('unzip -o "TAVI_STRESS_CLF_PARALLEL_v3.zip" > /dev/null')

0

# Load Data

In [4]:
DATA_DIR = "/content/Stress/"
train_X1 = np.load(DATA_DIR + "train_X1.npy")
train_X2 = np.load(DATA_DIR + "train_X2.npy")
train_X3 = np.load(DATA_DIR + "train_X3.npy")
train_Y = np.load(DATA_DIR + "train_y.npy")
test_X1 = np.load(DATA_DIR + "test_X1.npy")
test_X2 = np.load(DATA_DIR + "test_X2.npy")
test_X3 = np.load(DATA_DIR + "test_X3.npy")
test_Y = np.load(DATA_DIR + "test_y.npy")

# Config

In [5]:
IMG_SIZE = 224
BATCH_SIZE = 4
INITIAL_LEARNING_RATE = 0.0001
INITIAL_EPOCH = 300
EPOCH_PATIENCE = 30
FINE_TUNE_EPOCH = 0
TOTAL_EPOCH = INITIAL_EPOCH + FINE_TUNE_EPOCH

# Model Architecture

In [6]:
inputs = []
features = []

base_model = tf.keras.applications.ResNet50(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

for i in range(3):
    inp = tf.keras.Input(shape=(256, 256, 3))
    x = tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE)(inp)
    x = tf.keras.applications.resnet.preprocess_input(x)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    inputs.append(inp)
    features.append(x)

x = tf.keras.layers.concatenate(features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=inputs, outputs=x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 resizing (Resizing)         (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                              

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LEARNING_RATE),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    [train_X1, train_X2, train_X3],
    train_Y,
    batch_size=BATCH_SIZE,
    epochs=INITIAL_EPOCH,
    validation_data=([test_X1, test_X2, test_X3], test_Y),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=EPOCH_PATIENCE,
            restore_best_weights=True
        )
    ]
)

Epoch 1/300
108/108 [==============================] - 35s 105ms/step - loss: 1.1398 - accuracy: 0.5556 - val_loss: 0.6555 - val_accuracy: 0.6250
Epoch 2/300
108/108 [==============================] - 8s 71ms/step - loss: 0.7419 - accuracy: 0.5995 - val_loss: 0.6562 - val_accuracy: 0.5833
Epoch 3/300
108/108 [==============================] - 8s 77ms/step - loss: 0.6730 - accuracy: 0.6273 - val_loss: 0.6502 - val_accuracy: 0.6250
Epoch 4/300
108/108 [==============================] - 9s 85ms/step - loss: 0.6589 - accuracy: 0.6597 - val_loss: 0.6491 - val_accuracy: 0.6042
Epoch 5/300
108/108 [==============================] - 9s 82ms/step - loss: 0.6512 - accuracy: 0.6481 - val_loss: 0.6547 - val_accuracy: 0.6250
Epoch 6/300
108/108 [==============================] - 9s 84ms/step - loss: 0.6403 - accuracy: 0.6620 - val_loss: 0.6491 - val_accuracy: 0.6250
Epoch 7/300
108/108 [==============================] - 8s 74ms/step - loss: 0.6195 - accuracy: 0.6782 - val_loss: 0.6461 - val_accurac

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Fine Tuning

In [ ]:
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

model.compile(
    loss="binary_crossentropy",
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=INITIAL_LEARNING_RATE/10),
    metrics=["accuracy"]
)

history_fine = model.fit(
    [train_X1, train_X2, train_X3],
    train_Y,
    batch_size=BATCH_SIZE,
    epochs=TOTAL_EPOCH,
    initial_epoch=history.epoch[-1],
    validation_data=([test_X1, test_X2, test_X3], test_Y),
    # callbacks=[
    #     tf.keras.callbacks.EarlyStopping(
    #         monitor="val_loss",
    #         patience=EPOCH_PATIENCE,
    #         restore_best_weights=True
    #     )
    # ]
)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([INITIAL_EPOCH-1,INITIAL_EPOCH-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([INITIAL_EPOCH-1,INITIAL_EPOCH-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# Evaluation

In [ ]:
pred_y = model.predict([test_X1, test_X2, test_X3])
pred_y = np.round(pred_y).astype(int).reshape(-1)
test_y = test_Y.astype(int)

print(classification_report(test_y, pred_y))

cm = confusion_matrix(test_y, pred_y)
sns.heatmap(cm, annot=True, fmt="d")
